In [1]:
from feed_forward_neural_network import FeedForwardRegressor, FeedForwardClassifier

# Simplest example

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import r2_score, accuracy_score
from scipy.special import expit as logistic_func

### Create data from a regression model

In [3]:
# Simple Regression model
n_train_samples, n_test_samples, n_features = 100, 100, 10
theta = np.random.uniform(size = n_features) #regression coefficients
X = np.random.normal(size = (n_train_samples + n_test_samples, n_features)) #observations
xi = np.random.normal(size = n_train_samples + n_test_samples) #noise
y = X @ theta + xi #target
X_train, X_test, y_train, y_test = tts(X, y, train_size = n_train_samples)

### Create instance of FeedForwardRegressor

In [4]:
reg = FeedForwardRegressor()

### Fit training set

In [5]:
reg = reg.fit(X_train, y_train)

### Predict test set

In [6]:
y_pred = reg.predict(X_test)

### Evaluate prediction

In [7]:
print(r2_score(y_test, y_pred), reg.score(X_test, y_test))

0.6290636305417853 0.6290636305417853


### Clean up model

In [8]:
reg.delete_model_weights()  #else gpu memory leaks
del reg

## Same for binary classification

In [9]:
# Simple logistic model
n_train_samples, n_test_samples, n_features = 100, 100, 10
theta = np.random.uniform(size = n_features)
X = np.random.normal(size = (n_train_samples + n_test_samples, n_features))
xi = np.random.normal(size = n_train_samples + n_test_samples)
y = (logistic_func(X @ theta + xi) > 0.5).astype(int)
X_train, X_test, y_train, y_test = tts(X, y, train_size = n_train_samples)

# Learning
clf = FeedForwardClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:,1]
y_decision = clf.decision_function(X_test) #y_proba = logistic_func(y_decision)

print(accuracy_score(y_test, y_pred), 
      accuracy_score(y_test, (y_proba > 0.5).astype(float)),
      accuracy_score(y_test, (y_decision > 0.).astype(float)),
      clf.score(X_test, y_test))
clf.delete_model_weights()  #else gpu memory leaks
del clf

0.72 0.72 0.72 0.72


## Same for multiclass classification

In [10]:
# Simple hardmax model
n_train_samples, n_test_samples, n_features, n_classes = 100, 100, 10, 3
theta = np.random.uniform(size = (n_features, n_classes))
X = np.random.normal(size = (n_train_samples + n_test_samples, n_features))
xi = np.random.normal(size = (n_train_samples + n_test_samples, n_classes))
y = np.argmax(X @ theta + xi, axis = 1)
X_train, X_test, y_train, y_test = tts(X, y, train_size = n_train_samples)

# Learning
clf = FeedForwardClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)
y_decision = clf.decision_function(X_test) #y_proba = softmax(y_decision)

print(accuracy_score(y_test, y_pred), 
      accuracy_score(y_test, np.argmax(y_proba, axis = 1)),
      accuracy_score(y_test, np.argmax(y_decision, axis = 1)),
      clf.score(X_test, y_test))
clf.delete_model_weights()  #else gpu memory leaks
del clf

0.6 0.6 0.6 0.6


# Compare with Random Forests on a real-world dataset

In [11]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor as RF

In [12]:
seed = 0

### Load California Housing

In [13]:
X, y = fetch_california_housing(return_X_y = True)
X, y = StandardScaler().fit_transform(X), (y - y.mean())/y.std() #standardize data
train_samples_ratio = 0.8
X_train, X_test, y_train, y_test = tts(X, y, train_size = train_samples_ratio, random_state = seed)

### Train and evaluate a Random Forest

In [14]:
reg = RF(random_state = seed)
reg.fit(X_train, y_train)
print("RF", "R2-score :", reg.score(X_test, y_test))
del reg

RF R2-score : 0.7982270387164677


### Train and evaluate a FFNN with default parameters

In [15]:
reg = FeedForwardRegressor(random_state = seed)
reg.fit(X_train, y_train)
print("FFNN", "R2-score :", reg.score(X_test, y_test))
reg.delete_model_weights()  #else gpu memory leaks
del reg

FFNN R2-score : 0.2189434908152278


# Try other architectures and hyper-parameters

### Use dictionnaries of parameters provided in parameters_examples.py

In [16]:
from parameters_examples import mlp, glu, snn, mlpbatch #see parameters_examples.py for details and other examples
print("mlp", ":", mlp, "\n")
print("glu", ":", glu, "\n")
print("snn", ":", snn, "\n")
print("mlpbatch", ":", mlpbatch)

mlp : {'lr_scheduler': 'OneCycleLR', 'lr_scheduler_params': {'max_lr': 0.01, 'total_steps': 200}, 'max_iter': 200, 'learning_rate': 0.001, 'hidden_nn': <class 'architectures.DenseLayers'>, 'hidden_params': {'width': 512, 'depth': 2, 'dropout': 0.2, 'batch_norm': True}} 

glu : {'lr_scheduler': 'OneCycleLR', 'lr_scheduler_params': {'max_lr': 0.01, 'total_steps': 500}, 'max_iter': 500, 'learning_rate': 0.001, 'hidden_nn': <class 'architectures.GLULayers'>, 'hidden_params': {'width': 512, 'depth': 3, 'dropout': 0.2, 'batch_norm': True}} 

snn : {'lr_scheduler': 'OneCycleLR', 'lr_scheduler_params': {'max_lr': 0.01, 'total_steps': 500}, 'max_iter': 500, 'learning_rate': 0.001, 'hidden_nn': <class 'architectures.DenseLayers'>, 'hidden_params': {'width': 512, 'depth': 2, 'activation': 'SELU', 'initializer_params': {'gain_type': 'linear'}}} 

mlpbatch : {'lr_scheduler': 'OneCycleLR', 'lr_scheduler_params': {'max_lr': 0.01, 'total_steps': 200}, 'max_iter': 200, 'epochs': True, 'max_runtime': 36

### Pick one model

In [17]:
#mlpbatch is more appropriate for medium to large datasets, see parameters_examples.py for details
settings = "mlpbatch"
kwargs = eval(settings)
print(kwargs)

{'lr_scheduler': 'OneCycleLR', 'lr_scheduler_params': {'max_lr': 0.01, 'total_steps': 200}, 'max_iter': 200, 'epochs': True, 'max_runtime': 3600, 'learning_rate': 0.001, 'hidden_nn': <class 'architectures.DenseLayers'>, 'hidden_params': {'width': 512, 'depth': 2, 'dropout': 0.2, 'batch_norm': True}}


### Add data-specific infos (number of features, number of neurons on output layer)

In [18]:
multiclass = False
output = len(set(y_train)) if multiclass else 1 # number of neurons on output layer
kwargs["hidden_params"].update({"n_features" : X_train.shape[1], "output" : output})

### Pass the parameters during the initialization of the regressor

In [19]:
reg = FeedForwardRegressor(random_state = seed, **kwargs)

### Train and evaluate

In [20]:
reg.fit(X_train, y_train)
print(settings, "R2-score :", reg.score(X_test, y_test))
reg.delete_model_weights()  #else gpu memory leaks
del reg

mlpbatch R2-score : 0.8074418350336218


# Perform meta-learning.

### Train n = 10 models with different seeds.

In [21]:
n_learners = 10

settings = "mlpbatch"
kwargs = eval(settings)
multiclass = False
output = len(set(y_train)) if multiclass else 1 # number of neurons on output layer
kwargs["hidden_params"].update({"n_features" : X_train.shape[1], "output" : output})

predictions = np.zeros((n_learners, len(X_test)))
records = {}
for i in range(n_learners):
    reg = FeedForwardRegressor(random_state = i, **kwargs)
    reg.fit(X_train, y_train)
    records[i] = reg.record
    predictions[i] = reg.predict(X_test).reshape(-1)
    reg.delete_model_weights()  #else gpu memory leaks
    del reg

### Perform model ensembling

In [22]:
ensemble_prediction = predictions.mean(axis = 0)
print("Ensemble", "R2-score :", r2_score(y_test, ensemble_prediction))

Ensemble R2-score : 0.8082622719159248


### Perform model selection (seed picking)

In [23]:
best_seed = np.argmax([np.max(records[i]["validation"]) for i in range(n_learners)])
selected_prediction = predictions[best_seed]
print("Selection", "R2-score :", r2_score(y_test, selected_prediction))

Selection R2-score : 0.8014619989945647


# Use your own architecture on a dataset of tensor observations (eg: MNIST)

### Define a pytorch module
You can take inspiration from the architectures.py file. Here we will just copy [the pytorch example.](https://github.com/pytorch/examples/blob/master/mnist/main.py)

In [24]:
import torch
import importlib
if importlib.util.find_spec('torch.cuda'):
    device = "cuda" if torch.cuda.is_available() else "cpu"
else:
    device = "cpu"

class BasicConvNet(torch.nn.Module): 
    def __init__(self, device = device):
        super(BasicConvNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 32, 3, 1, device = device)
        self.conv2 = torch.nn.Conv2d(32, 64, 3, 1, device = device)
        self.dropout1 = torch.nn.Dropout(0.25)
        self.dropout2 = torch.nn.Dropout(0.5)
        self.fc1 = torch.nn.Linear(9216, 128, device = device)
        self.fc2 = torch.nn.Linear(128, 10, device = device)
        
    def forward(self, x):
        x = self.conv1(x)
        x = torch.nn.functional.relu(x)
        x = self.conv2(x)
        x = torch.nn.functional.relu(x)
        x = torch.nn.functional.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = torch.nn.functional.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

### Load MNIST

In [25]:
from sklearn.datasets import fetch_openml
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
train_samples_ratio = 0.8
X_train, X_test, y_train, y_test = tts(X, y, train_size = train_samples_ratio, stratify = y)
X_train, X_test = X_train.reshape((-1,1,28,28)), X_test.reshape((-1,1,28,28)) #reshape as a tensor

### Pick training hyper-parameters
We will just copy [the pytorch example.](https://github.com/pytorch/examples/blob/master/mnist/main.py)

In [26]:
kwargs = {"optimizer" : "Adadelta",
        "validation_fraction" : False,
        "early_stopping_criterion" : False,    
        "learning_rate" : 1.0,
        "optimizer_params" : {},
        "lr_scheduler": "StepLR",
        "lr_scheduler_params":{"step_size" : 1,"gamma" : 0.7},
        "batch_size": 64,
        "max_iter" : 14 * int(len(y_train)/64), #14 epochs with batchsize 64
        "hidden_nn" : BasicConvNet,
        "hidden_params" : {}}

### Fit, predict and evaluate

In [27]:
clf = FeedForwardClassifier(**kwargs)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
clf.delete_model_weights()  #else gpu memory leaks
del clf

0.9895714285714285


# Play with all the training parameters

In [28]:
#Exhaustive parameter list:
if False:
    if False:
        hidden_nn, #architecture of hidden layers, torch.nn.Module (if False, uses default mlp architecture).
        hidden_params, #hyper_parameters for hidden_nn initialization
                 
        #learning parameters
        default_loss_function, #set by the subclass
        optimizer, #a torch optimizer, str
        learning_rate, #float
        optimizer_params,#except learning rate, dict
        lr_scheduler, #a torch optimizer.lr_scheduler, str
        lr_scheduler_params,#dict
        batch_size, #if None or False: full batch, if int number of samples, if float share of samples
                 
        #convergence parameters
        max_iter, #iterations, not epochs (epochs = max_iter/batch_size), int
        epochs, #max_iter => max_iter * train_size / batch_size, bool
        max_runtime, #unprecise, float or int
        validation_fraction, #if None or False: no validation, if int number of samples, if float share of samples
        should_stratify, #validation split strategy, bool
        early_stopping_criterion, #either "loss" or "validation", str
        convergence_tol, #if None or False: always max_iter, else float 
        divergence_tol, #if None or False: always max_iter, else float 
        
        #AdaCap Parameters
        #Tikhonov operator specific parameters
        adacap,
        closeform_parameter_init, #if None or False: regular FFNN, if int or float lambda initial value, if "max_variation" or "min_value" grid-search
        closeform_intercept,#add unitary feature to covar matrix, bool
                 
        #MuddlingLabelRegularization specific parameters
        n_permut, #if int number of permutations, if None or False no permutations
        permutation_scale, #weight of permutation term added to the loss, float
                 
        #MLR additional regularization techniques
        dithering_scale,#if float dithering white noise standard-deviation, if None or False no gaussian dithering
        target_rotation_scale,#if float dithering structured noise standard-deviation, if None or False no structured noise dithering
        
        #Target handling
        center_target,#center target around mean (behaves differently for binary clf), bool 
        rescale_target,#divide target by std before fitting, bool 
        loss_imbalance,#smaller weights on majority classes, bool
                 
        random_state, #scikit-learn random state, will also set torch generator using a different seed
        release_train_memory, #delete validation samples, optimizer and lr scheduler after training
        save_repository,#repository in which tempory models will be saved (see PATH and REP)
        verbose #if False mute, if True print at each iteration, if int print if iter%verbose == 0

# Analysis

In [29]:
X, y = fetch_california_housing(return_X_y = True)
X, y = StandardScaler().fit_transform(X), (y - y.mean())/y.std() #standardize data
train_samples_ratio = 0.8
X_train, X_test, y_train, y_test = tts(X, y, train_size = train_samples_ratio)

### Print metrics values every 10 iterations

In [30]:
reg = FeedForwardRegressor(release_train_memory = False, verbose = 10)
reg = reg.fit(X_train, y_train)

|     iter |     loss |     time | validati |
|        0 | 1.83e+00 | 7.30e-03 | -2.86e-01 |
|       10 | 1.23e+00 | 3.37e-03 | 1.02e-01 |
|       20 | 8.92e-01 | 3.35e-03 | 5.62e-01 |
|       30 | 8.33e-01 | 3.42e-03 | 5.59e-01 |
|       40 | 7.56e-01 | 3.10e-03 | 6.38e-01 |
|       50 | 7.77e-01 | 5.33e-03 | 6.72e-01 |


### Save learning dynamic

In [31]:
record = reg.record
print(record.keys()) # you can then plot these with matplotlib

dict_keys(['loss', 'time', 'validation'])


### Hidden layers

In [32]:
hidden_layers = reg.hidden_layers
print(hidden_layers)
del hidden_layers

DefaultDenseLayers(
  (model): Sequential(
    (0): Linear(in_features=8, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.2, inplace=False)
    (7): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Linear(in_features=256, out_features=1, bias=True)
  )
)


### Optimizer and Scheduler

In [35]:
optimizer = reg.optimizer_instance
print(optimizer)
if not reg.cst_lr:
    lr_scheduler = reg.lr_scheduler_instance
    print(lr_scheduler)

Adam (
Parameter Group 0
    amsgrad: False
    base_momentum: 0.85
    betas: (0.949798714699762, 0.999)
    eps: 1e-08
    initial_lr: 0.0004
    lr: 2.0168449509684565e-05
    max_lr: 0.01
    max_momentum: 0.95
    min_lr: 4e-08
    weight_decay: 0
)


### Re-use a network copy elsewhere

In [36]:
import torch
storage_path = reg.save_repository + reg.PATH #this file will be deleted with reg.delete_model_weights()
network_copy = torch.load(storage_path)
new_storage_path = "./saved_model.pt"
torch.save(network_copy, new_storage_path)

### Clean up

In [37]:
reg._release_train_memory()
reg.delete_model_weights()  #else gpu memory leaks
del reg

In [38]:
import os
os.remove(new_storage_path)

# Remarks on batch-learning.
There are two ways in which the default behavior of the FeedForwardNeuralNetwork differs from the standard approach. Note however that in both cases changing parameters is enough to fall back to the most classical setting. 

- The FeedForwardNeuralNetwork class can perform batch-learning but by default, it tries to avoid doing so (which is most often the correct strategy for small tabular datasets). However, to avoid GPU memory errors, some caps on the maximum size of weights and activations matrices are hard-coded: the width and height is at most 4096. This means that even if the parameter "batch_size" is equal to False, batch-learning will be used if the training set (substracting the validation set if any) is larger than 4096. Mini-batches of size smaller than 4096 can still be used by setting the "batch_size" to an int.\\


- The cap on the maximum number of iterations corresponds by default to the maximum number of gradient updates, not epochs. This enforces a soft constraint on the maximum training time with respect to the training set size, as at most, the gradients will be computed for max_iter times 4096 observations. It is possible nonetheless to set the maximum number of epochs instead, by setting the parameter "epochs" as True.